In [1]:
import numpy as np

from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

from nsopy import SGMDoubleSimpleAveraging as DSA
from nsopy.loggers import EnhancedDualMethodLogger

output_notebook()
%cd ..

from smpspy.oracles import TwoStage_SMPS_InnerProblem

Loading BokehJS ...

C:\Users\vujanicr\PycharmProjects\smpspy_base


# Solving dual model using DSA with Entropy Prox Term

Instantiating inner problem

### Solve battery of problems

In [2]:
# Setup
BENCHMARKS_PATH = './smpspy/benchmark_problems/2_caroe_schultz/'

n_S_exp = [10, 50, 100, 500]
N_STEPS = 200
GAMMA = 1.0

In [3]:
# First generate traditional DSA

inner_problems = {}
methods = {}
method_loggers = {}

for n_S in n_S_exp:
    ip = TwoStage_SMPS_InnerProblem(BENCHMARKS_PATH+'caroe_schultz_{}'.format(n_S))
    dsa = DSA(ip.oracle, ip.projection_function, dimension=ip.dimension, gamma=GAMMA)
    logger_dsa = EnhancedDualMethodLogger(dsa)
    
    inner_problems[n_S] = ip
    methods[n_S] = dsa
    method_loggers[n_S] = logger_dsa

Parsing nominal model information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10.cor and .tim ...
Parsing stochastic information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10.sto ...
Stochastic model is of type SCENARIOS DISCRETE
Parsing nominal model information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_50.cor and .tim ...
Parsing stochastic information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_50.sto ...
Stochastic model is of type SCENARIOS DISCRETE
Parsing nominal model information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_100.cor and .tim ...
Parsing stochastic information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_100.sto ...
Stochastic model is of type SCENARIOS DISCRETE
Parsing nominal model information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_500.cor and .tim ...
Parsing stochastic information from ./smpspy/benchmark_problems/2_caroe_s

In [4]:
for n_S, method in methods.items():
    for step in range(N_STEPS):
        if not step % 100:
            print('[n_S={}] step: {} of method {}'.format(n_S, str(step), str(method.desc)))
        method.dual_step()

[n_S=500] step: 0 of method DSA, $\gamma = 1.0$
[n_S=500] step: 100 of method DSA, $\gamma = 1.0$
[n_S=10] step: 0 of method DSA, $\gamma = 1.0$
[n_S=10] step: 100 of method DSA, $\gamma = 1.0$
[n_S=100] step: 0 of method DSA, $\gamma = 1.0$
[n_S=100] step: 100 of method DSA, $\gamma = 1.0$
[n_S=50] step: 0 of method DSA, $\gamma = 1.0$
[n_S=50] step: 100 of method DSA, $\gamma = 1.0$


In [5]:
inner_problems_entropy = {}
methods_entropy = {}
method_loggers_entropy = {}

for n_S in n_S_exp:
    R_a_posteriori = np.linalg.norm(methods[n_S].lambda_k, ord=np.inf)
    R_safe = R_a_posteriori*1.1
    
    ip = TwoStage_SMPS_InnerProblem(BENCHMARKS_PATH+'caroe_schultz_{}'.format(n_S), R=R_safe)
    dsa_entropy = DSA(ip.oracle, ip.softmax_projection, dimension=ip.dimension, gamma=GAMMA)
    logger_dsa_entropy = EnhancedDualMethodLogger(dsa_entropy)
    
    inner_problems_entropy[n_S] = ip
    methods_entropy[n_S] = dsa_entropy
    method_loggers_entropy[n_S] = logger_dsa_entropy

Parsing nominal model information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10.cor and .tim ...
Parsing stochastic information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10.sto ...
Stochastic model is of type SCENARIOS DISCRETE
Parsing nominal model information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_50.cor and .tim ...
Parsing stochastic information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_50.sto ...
Stochastic model is of type SCENARIOS DISCRETE
Parsing nominal model information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_100.cor and .tim ...
Parsing stochastic information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_100.sto ...
Stochastic model is of type SCENARIOS DISCRETE
Parsing nominal model information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_500.cor and .tim ...
Parsing stochastic information from ./smpspy/benchmark_problems/2_caroe_s

In [6]:
for n_S, method in methods_entropy.items():
    for step in range(N_STEPS):
        if not step % 100:
            print('[n_S={}] step: {} of method {}'.format(n_S, str(step), str(method.desc)))
        method.dual_step()

[n_S=500] step: 0 of method DSA, $\gamma = 1.0$
[n_S=500] step: 100 of method DSA, $\gamma = 1.0$
[n_S=10] step: 0 of method DSA, $\gamma = 1.0$
[n_S=10] step: 100 of method DSA, $\gamma = 1.0$
[n_S=100] step: 0 of method DSA, $\gamma = 1.0$
[n_S=100] step: 100 of method DSA, $\gamma = 1.0$
[n_S=50] step: 0 of method DSA, $\gamma = 1.0$
[n_S=50] step: 100 of method DSA, $\gamma = 1.0$


In [7]:
# find "d*"
d_stars = {}
EPS = 0.01

for n_S in n_S_exp:
    d_star_dsa = max(method_loggers[n_S].d_k_iterates)
    d_star_dsa_entropy = max(method_loggers_entropy[n_S].d_k_iterates)
    d_stars[n_S] = max(d_star_dsa, d_star_dsa_entropy) + EPS

In [8]:
p = figure(title="comparison", x_axis_label='iteration', y_axis_label='d* - d_k', y_axis_type='log', toolbar_location='above')

plot_colors = {
    10: 'blue',
    50: 'green',
    100: 'red',
    500: 'orange',
    1000: 'purple',
}

In [9]:
for n_S in n_S_exp:
    logger = method_loggers[n_S]
    p.line(range(len(logger.d_k_iterates)), d_stars[n_S] - np.array(logger.d_k_iterates), legend="DSA, n_scen={}, gamma={}".format(n_S, GAMMA, inner_problems[n_S].R), 
           color=plot_colors[n_S], line_dash='dashed')
    
for n_S in n_S_exp:
    logger = method_loggers_entropy[n_S]
    p.line(range(len(logger.d_k_iterates)), d_stars[n_S] - np.array(logger.d_k_iterates), legend="DSA Entropy, n_scen={}, gamma={}, R={}".format(n_S, GAMMA, inner_problems_entropy[n_S].R), 
           color=plot_colors[n_S])

In [10]:
p.legend.location = "top_right"
p.legend.visible = True
p.legend.background_fill_alpha = 0.5
show(p)

### Single run

In [3]:
ip = TwoStage_SMPS_InnerProblem('./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10')

Parsing nominal model information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10.cor and .tim ...
Parsing stochastic information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10.sto ...
Stochastic model is of type SCENARIOS DISCRETE


First solving it with DSA

In [4]:
GAMMA = 1.0

dsa = DSA(ip.oracle, ip.projection_function, dimension=ip.dimension, gamma=GAMMA)
logger_dsa = EnhancedDualMethodLogger(dsa)

In [5]:
for iteration in range(1000):
    if not iteration%50:
        print('Iteration: {}, d_k={}'.format(iteration, dsa.d_k))
    dsa.dual_step()

Iteration: 0, d_k=-inf
Iteration: 50, d_k=-55.0159186353
Iteration: 100, d_k=-54.9786759617
Iteration: 150, d_k=-54.8958271191
Iteration: 200, d_k=-54.946123664
Iteration: 250, d_k=-54.9126206845
Iteration: 300, d_k=-54.8681531927
Iteration: 350, d_k=-54.8759044684
Iteration: 400, d_k=-54.83683095
Iteration: 450, d_k=-54.8435540524
Iteration: 500, d_k=-54.833795125
Iteration: 550, d_k=-54.8560302563
Iteration: 600, d_k=-54.8279813457
Iteration: 650, d_k=-54.8655052529
Iteration: 700, d_k=-54.8348583158
Iteration: 750, d_k=-54.8305845772
Iteration: 800, d_k=-54.8340387889
Iteration: 850, d_k=-54.8374382962
Iteration: 900, d_k=-54.8204466776
Iteration: 950, d_k=-54.8407756352
Iteration: 1000, d_k=-54.8468588731
Iteration: 1050, d_k=-54.8219773445
Iteration: 1100, d_k=-54.8287186107
Iteration: 1150, d_k=-54.8173967699
Iteration: 1200, d_k=-54.8214714153
Iteration: 1250, d_k=-54.8338867381
Iteration: 1300, d_k=-54.8255492724
Iteration: 1350, d_k=-54.8146888169
Iteration: 1400, d_k=-54.8334

Then get the required parameters (R is derived a posteriori)

In [23]:
R_a_posteriori = np.linalg.norm(dsa.lambda_k, ord=np.inf)
R_safe = R_a_posteriori*1.1
ip = TwoStage_SMPS_InnerProblem('./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10', R=R_safe)

print('A-posteriori R={}'.format(R_a_posteriori))

Parsing nominal model information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10.cor and .tim ...
Parsing stochastic information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10.sto ...
Stochastic model is of type SCENARIOS DISCRETE
A-posteriori R=0.994113159479


Solve it using DSA with Entropy prox function. **Note that the only difference is that we pass in softmax projection function!**

In [24]:
dsa_entropy = DSA(ip.oracle, ip.softmax_projection, dimension=ip.dimension, gamma=GAMMA)
logger_dsa_entropy = EnhancedDualMethodLogger(dsa_entropy)

In [25]:
for iteration in range(1000):
    if not iteration%50:
        print('Iteration: {}, d_k={}'.format(iteration, dsa_entropy.d_k))
    dsa_entropy.dual_step()

Iteration: 0, d_k=-inf
Iteration: 50, d_k=-55.7332142332
Iteration: 100, d_k=-55.567744575
Iteration: 150, d_k=-55.523640381
Iteration: 200, d_k=-55.3948346217
Iteration: 250, d_k=-55.2914856945
Iteration: 300, d_k=-54.9578756746
Iteration: 350, d_k=-54.9153649334
Iteration: 400, d_k=-55.0115510287
Iteration: 450, d_k=-54.9759676722
Iteration: 500, d_k=-54.9421228308
Iteration: 550, d_k=-54.9955055834
Iteration: 600, d_k=-54.9981796194
Iteration: 650, d_k=-54.911199455
Iteration: 700, d_k=-54.975455558
Iteration: 750, d_k=-54.9287581365
Iteration: 800, d_k=-54.9192514909
Iteration: 850, d_k=-54.8836071148
Iteration: 900, d_k=-54.8820872471
Iteration: 950, d_k=-54.8604667678
Iteration: 1000, d_k=-54.8862314468
Iteration: 1050, d_k=-54.8452524481
Iteration: 1100, d_k=-54.8691533025
Iteration: 1150, d_k=-54.8887367985
Iteration: 1200, d_k=-54.8745224513
Iteration: 1250, d_k=-54.8614982156
Iteration: 1300, d_k=-54.8558172355
Iteration: 1350, d_k=-54.8451513578
Iteration: 1400, d_k=-54.8543

In [26]:
logger_dsa.lambda_k_iterates[-1]

array([ 0.14400903, -0.24433813,  0.27291032, -0.20353668,  0.27981351,
       -0.40031441, -0.15550504,  0.99409693, -0.15900846, -0.09654058,
        0.148661  , -0.40031441, -0.19887074,  0.99409693, -0.17734202,
       -0.39408986, -0.15466759, -0.24905981])

In [27]:
logger_dsa_entropy.lambda_k_iterates[-1]

array([ 0.07751982,  0.01277697,  0.4409704 , -0.31506227,  0.46902291,
       -0.39622856, -0.28952264,  1.0066311 , -0.15609555, -0.10477626,
        0.22667051, -0.39622856, -0.23482028,  1.00006845, -0.26926575,
       -0.40409742, -0.26447941, -0.40308344])

In [28]:
p = figure(title="comparison", x_axis_label='iteration', y_axis_label='d_k')

In [29]:
p.line(range(len(logger_dsa.d_k_iterates)), logger_dsa.d_k_iterates, legend="DSA, gamma={}".format(GAMMA, R_safe))
p.line(range(len(logger_dsa_entropy.d_k_iterates)), logger_dsa_entropy.d_k_iterates, legend="DSA Entropy, gamma={}, R={}".format(GAMMA, R_safe), color='red')

GlyphRenderer(id='4c953611-3cb0-4a7c-87e1-95bae7ae6067', ...)

In [30]:
p.legend.location = "bottom_right"
show(p)